In [1]:
import scala.io.Source

Intitializing Scala interpreter ...

Spark Web UI available at http://172.27.224.1:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1662839384320)
SparkSession available as 'spark'


import scala.io.Source


In [3]:
type Num = Int
final val ZERO = 0

case class Options(set: Set[Num])
case class Cell(value: Num, options: Options)
case class Grid(array: Array[Array[Cell]])
case class Position(row: Int, col: Int)
case class Pair(cell: Cell, position: Position)
trait Strategy {
  def apply(grid: Grid): Grid
}

object Sudoku {
  val size = 9
  val numbers: Array[Num] = Array[Num](1 to 9 :_*)
}

defined type alias Num
ZERO: Int(0) = 0
defined class Options
defined class Cell
defined class Grid
defined class Position
defined class Pair
defined trait Strategy
defined object Sudoku


In [5]:
class Sudoku(val size: Int) {
  def this() = this(Sudoku.size)
  def readFile(src: String): Array[Array[Int]] =
    Source.fromFile(src)
      .map(_.getNumericValue)
      .filter(e => e != -1)
      .toArray
      .grouped(size)
      .toArray

  def arrayToGrid(a: Array[Array[Int]]): Grid =
    Grid(a.map(r => r.map(c => Cell(c, Options(Set())))))
}

defined class Sudoku
Companions must be defined together; you may wish to use :paste mode for this.


In [10]:
implicit class ArrayNum(val a: Array[Num]) {
  def difference(b: Array[Num]): Array[Num] =
    a.filter(n => !b.contains(n)).filter(n => n != ZERO)
}

defined class ArrayNum


In [13]:
implicit class ArrayCell(val a: Array[Cell]) {
  def getValues: Array[Num] = a.map(cell => cell.value).filter(e => e != ZERO)
  def getOptions: Array[Num] = a.map(cell => cell.options.set).reduce(_ union _).toArray
  def union(b: Array[Cell]): Array[Cell] =
    (a ++ b).distinct
}

defined class ArrayCell


In [14]:
implicit class ArraPair(a: Array[Pair]) {
  def valueEqualTo(n: Num): Array[Pair] = a.filter(cell => cell.cell.value == n)
  def index(): Array[Position] = a.map(_.position)
  def col(col: Int): Array[Pair] = a.filter(_.position.col == col)
  def row(row: Int): Array[Pair] = a.filter(_.position.row == row)
  def box(num: Int): Array[Pair] = {
    val row = (num / 3) * 3
    val col = (num % 3) * 3
    a.filter(p => p.position.row >= row && p.position.row < row + 3 && p.position.col >= col && p.position.col < col + 3)
  }
  def uniqueOption(): Array[Pair] = a.filter(pair => pair.cell.options.set.size == 1)
  /*def pair(): Array[Pair] =
    val pair = a.filter(pair => pair.cell.options.set.size == 2)
    if pair.length == 2 then pair else Array()*/

  def emptyCells(): Array[Pair] = a.filter(_.cell.value == ZERO)
  def groupBy(): List[Map[Num, Array[Pair]]] =
    a.flatMap(e => e.cell.options.set.map(i => (i, e)))
    .groupBy(e => e(0))
    .map(e => Map(e._1 -> e._2.map(e => e._2)))
    .toList
  def groupByOption(): List[Map[Num, Array[Position]]] =
    a.flatMap(e => e.cell.options.set.map(i => (i, e.position)))
    .groupBy(e => e(0))
    .map(e => Map(e._1 -> e._2.map(e => e._2)))
    .toList
  def groupByPosition(): List[Map[Position, Array[Num]]] =
    a.flatMap(e => e.cell.options.set.map(i => (i, e.position)))
    .groupBy(e => e(1))
    .map(e => Map(e._1 -> e._2.map(e => e._1)))
    .toList
}


<console>: 52: error: (Num, Pair) does not take parameters